## I. Import Libraries and Data

In [ ]:
import re
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tpot import TPOTClassifier
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix, hstack
from scipy.sparse import csr_matrix, hstack

from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

## II. Define Functions

## A. Cleaning and Read Functions

In [ ]:
def cleanArticle(string):
    strip_special_chars = re.compile("[^A-Za-z0-9' ]+")
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def read_perez_dataset(dataset_name):
    
    def remove_numbers(in_str):
        return re.sub(r'[0-9]+', '', in_str)
    
    print("Reading dataset")
    result_data_list = []
    data_dir = PEREZ_DATASET_PATH
    for news_type in ['fake', 'legit']:
        folder = '%s/%s/%s' % (data_dir, dataset_name, news_type)
        for fname in os.listdir(folder):
            result_data = {}
            result_data['dataset_name'] = dataset_name
            result_data['news_type'] = news_type
            if news_type == 'fake':
                result_data['is_fake'] = 1
            else:
                result_data['is_fake'] = 0
            if dataset_name == 'fakeNewsDataset':
                result_data['news_category'] = remove_numbers(fname.split('.')[0])
            result_data['file_name'] = fname
            filepath = os.path.join(folder, fname)
            with open(filepath, 'r', encoding="utf8") as f:
                file_data = f.read().split('\n')
                # Some articles don't have a headline, but only article body.
                if len(file_data) > 1:
                    news_content_data = ' '.join(file_data[2:])
                    result_data['news_headline'] = file_data[0]
                else:
                    news_content_data = file_data[0]
                    result_data['news_headline'] = ''
                result_data['news_content'] = news_content_data
                result_data['news_all'] = ' '.join(file_data[0:])
                result_data_list.append(result_data)
                
    df = pd.DataFrame(result_data_list)
    
    df['news_all_clean'] = df['news_all'].apply(lambda a: cleanArticle(a))
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['is_fake',
                                                               'news_type','file_name'],
                                                               axis = 1), 
                                                        df['is_fake'], 
                                                        test_size=.2, random_state=RANDOM_SEED)
    
    print("Finished reading dataset")
    return df, X_train, y_train, X_test, y_test

def model_report(title, y_test, predictions, predictions_proba):

    """
    Output: Classification report, confusion matrix, and ROC curve
    """
    print(title)
    print("---------")
    print(classification_report(y_test, predictions))

    cm = metrics.confusion_matrix(y_test, predictions)
    plt.figure(figsize=(3,3))
    sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r');
    plt.ylabel('Actual label');
    plt.xlabel('Predicted label');
    all_sample_title = 'Accuracy: {0}'.format(round(metrics.accuracy_score(y_test, predictions),2))
    plt.title(all_sample_title, size = 15)
    plt.show()
    
    fpr, tpr, threshold = metrics.roc_curve(y_test, predictions_proba)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

## B. Linguistic-based feature creators

### 1. Ngram & Punctuation

In [ ]:
def pad_punct(s):
    """
    Add padding around specified punctuation.
    """
    s = re.sub('([.,!?():])', r' \1 ', s)
    s = re.sub('\s{2,}', ' ', s)
    return s
def tfidf_vectorizer_custom(train, test, ngram_range):
    """
    Create a tfidf vectorized set for train and test data that counts punctuation.
    Ngram range = (1,3)
    """
    train = train.apply(pad_punct)
    test = test.apply(pad_punct)
    vect = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b|!|\.|,|\)|\(|\:|\?|\"|\'", #pattern keep punctuation in vectorizer
                          ngram_range = ngram_range).fit(train)
    vocab = vect.vocabulary_.keys()
    vocab = sorted(vocab, key=len)
    print("Traing data info:")
    print('- Vocabulary len:', len(vect.get_feature_names()))
    print('- Longest phrase:', max(vect.vocabulary_, key=len))
    print('- Smallest 10 phrases:', vocab[0:10])
    print('- Sample of features:',np.array(vect.get_feature_names()))
    train_vectorized = vect.transform(train)
    test_vectorized = vect.transform(test)
    return train_vectorized, test_vectorized

### 2. Psycholinguistic features

In [ ]:
def LIWC_counter(df, df_LIWC):
    """
    Inputs: - Data set with field: news_all_clean
            - Preprocessed LIWC lookup table
    Ouput:  - Data set with 73 additional fields for LIWC count results (normalized over string word count)
    """
    LIWC_vars = train_LIWC.drop('Word', axis = 1).columns.values
    for i in LIWC_vars:
        df[i] = 0
        words = df_LIWC[df_LIWC[i] == 1]['Word']
        for a in words:
            df[i] = (df[i] + 
                df.news_all_clean.str.count(str(" " + a + " "))  + #free floating word
                df.news_all_clean.str.count(str("^" + a + " ")) +  #start word
                df.news_all_clean.str.count(str(" " + a + "$")))   #end word
        df[i] = df[i] / df.news_all_clean.str.count(" ") #normalize over word count
    return df

### 3. Readability

### 4. Syntax

## III. Read in Data, Define Test/Train Set

In [ ]:
RANDOM_SEED = 42
PEREZ_DATASET_PATH = "../data/fakeNewsDatasets_Perez-Rosas2018"
np.random.seed(RANDOM_SEED)
perez_full, train_data, train_labels, test_data, test_labels = read_perez_dataset('fakeNewsDataset')
train_data.head()

## IV. Run Train/Test Set through LIWC Rented License

We rented the LIWC dataset (for 30 days), as seen here: https://store5.esellerate.net/store/checkout/CustomLayout.aspx?s=STR6622550055&pc=&page=OnePageMoreInfo.htm&SkuRefNum=SKU48070077205

We downloaded the software, outputted the train/test set, ran the train/test set through the LIWC software, and uploaded the LIWC results.
The software marks words to 73 LIWC categories in a binary format (with an 'X')

In [ ]:
train_data['news_all'].to_csv("train_data_news_all_forLIWC.txt")
test_data['news_all'].to_csv("test_data_news_all_forLIWC.txt")
train_LIWC = pd.read_csv(
    "LIWC2015 Results (train_data_news_all_clean_forLIWC).csv").fillna(0).drop('Unnamed: 74', axis = 1).replace('X',1)
test_LIWC = pd.read_csv(
    "LIWC2015 Results (test_data_news_all_clean_forLIWC).csv").fillna(0).drop('Unnamed: 74', axis = 1).replace('X',1)

In [ ]:
train_LIWC.head()

In [ ]:
test_LIWC.head()

## IV. Quick Look at Data

In [ ]:
train_data['news_all_clean'].iloc[1]

In [ ]:
print("full perez size: ", perez_full.shape)
print("train size: ",train_labels.shape)

In [ ]:
sns.catplot(x="news_category", kind = "count", hue="is_fake", data=perez_full)
plt.show()

In [ ]:
perez_full.groupby(['news_category','is_fake']).size()

## V. Create Feature-Enriched Training/Testing Datasets

## A. Implement Feature Creating Functions

#### 1. Ngram & Punctuation

In [ ]:
train_data_vectorized, test_data_vectorized = tfidf_vectorizer_custom(
    train_data['news_all_clean'], test_data['news_all_clean'], ngram_range = (1,3))

#### 2. Psycholinguistic features

In [ ]:
train_data_LIWC = LIWC_counter(train_data, train_LIWC)
test_data_LIWC = LIWC_counter(test_data, test_LIWC)
train_data_LIWC.head()

### B. Unify All Features in Single Dataset

In [ ]:
def add_feature(sparse, df_to_sparse, df_ignore):
    """
    Returns sparse feature matrix with added feature.
    """
    return hstack([coo_matrix(df_to_sparse.drop(df_ignore.columns.values, axis = 1)),sparse])

In [ ]:
#1. Combine ngram and LIWC
train_ngram_LIWC = add_feature(train_data_vectorized, train_data_LIWC, train_data)
test_ngram_LIWC = add_feature(test_data_vectorized, test_data_LIWC, test_data)

#5. Add news category
vectorizer = CountVectorizer()
vectorizer.fit(train_data['news_category'])
train_news_category_vec = vectorizer.transform(train_data['news_category'])
test_news_category_vec  = vectorizer.transform(test_data['news_category'])
train_final = hstack([train_ngram_LIWC,train_news_category_vec])
test_final =  hstack([test_ngram_LIWC,test_news_category_vec])

In [ ]:
train_final

## V. Classification Models with Automated Machine Learning (TPOT)

#### 1. Run TPOT Optimizer

In [ ]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2,
                                   config_dict = "TPOT sparse")
#pipeline_optimizer.fit(train_final, train_labels) #This takes a couple hours to run

#### 2. Output TPOT Algorithm

In [ ]:
pipeline_optimizer.export('tpot_ngram_LIWC.py')

#### 3. Implement TPOT Algorithm

In [ ]:
from sklearn.ensemble import RandomForestClassifier

exported_pipeline = RandomForestClassifier(bootstrap=True, criterion="gini", 
                                           max_features=0.7000000000000001, 
                                           min_samples_leaf=19, 
                                           min_samples_split=5, 
                                           n_estimators=100)


exported_pipeline.fit(train_final, train_labels)
ORIG_results = exported_pipeline.predict(test_final)
ORIG_proba = exported_pipeline.predict_proba(test_final)
print("accuracy on original test set: ",np.mean(ORIG_results == test_labels))

#### 4. Output TPOT Model Report